In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string
import re
import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM, SimpleRNN, Dropout
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Sequential 

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
train_df = pd.read_csv("/content/train.txt", names =  ["Incorrect","Correct"],delimiter = ".\t")

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
test_df = pd.read_csv("/content/validation.txt", names = ['Incorrect', 'Correct'], delimiter = ".\t")

In [ ]:
df_correct = pd.DataFrame({"Text":[],
                           "Target":[]})
df_correct['Text'] = train_df['Correct']
df_correct['Target'] = 0
df_incorrect = pd.DataFrame({"Text":[],
                             "Target":[]})
df_incorrect['Text'] = train_df['Incorrect']
df_incorrect['Target'] = 1
df_train = pd.concat([df_correct, df_incorrect])


In [ ]:
df_train

,Text,Target
0,In my opinion the TV channels should show the ...,0
1,"There I began to study Family Therapy , discip...",0
2,"It seems to me , that in this song there are r...",0
3,"So , I want to say that I admire your right di...",0
4,I graduated from Chung university .,0
...,...,...
153177,This is my family on the picture,1
153178,I have lunch at noon and get home 2 p.m,1
153179,"Dear Mary , I listened the song that you spok...",1
153180,Look at my photos of my last holidays,1


In [ ]:
df_correct = pd.DataFrame({"Text":[],
                           "Target":[]})
df_correct['Text'] = test_df['Correct']
df_correct['Target'] = 0
df_incorrect = pd.DataFrame({"Text":[],
                             "Target":[]})
df_incorrect['Text'] = test_df['Incorrect']
df_incorrect['Target'] = 1
df_test = pd.concat([df_correct, df_incorrect])

In [ ]:
df_test

,Text,Target
0,In summer the people prefer to go to the beach...,0
1,It is common to see students at wherever place...,0
2,"First , I want to learn English very well , so...",0
3,I just went on an amazing trip . I took a flig...,0
4,"I like to walk in the park , in the street , n...",0
...,...,...
38291,My experience is in manager groups in a comput...,1
38292,My birthday is June 26th,1
38293,* Julie remembered that the visitors are goin...,1
38294,He was project manager for 2 years of the cust...,1


In [ ]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [ ]:
max_words = 50
df_train = df_train[df_train["Text"].apply(len) < max_words]

In [ ]:
#tokenize the data
VOCAB_SIZE = 20000
tokenizer = Tokenizer(num_words = VOCAB_SIZE)
tokenizer.fit_on_texts(df_train['Text'])

max_len = df_train['Text'].apply(len).max()

In [ ]:
#pad the train and test sequences after fitting on tokenizer
df_train['Text'] = tokenizer.texts_to_sequences(df_train['Text'])
x_train = pad_sequences(df_train['Text'],
                            maxlen=max_len,
                            padding='post')
y_train = df_train['Target']

In [ ]:
df_test['Text'] = tokenizer.texts_to_sequences(df_test['Text'])
x_test = pad_sequences(df_test['Text'],
                           maxlen=max_len,
                           padding='post')
y_test = df_test['Target']

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=VOCAB_SIZE,
                    output_dim=128,
                    input_length=max_len))
model.add(LSTM(50, activation='tanh'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
result = model.fit(x_train, y_train, batch_size=64, epochs=3)
#validation_data=(x_test, y_test)
#'AUC','Precision','Recall',

Epoch 1/3
1212/1212 [==============================] - 112s 85ms/step - loss: 8.0380 - accuracy: 0.4789
Epoch 2/3
1212/1212 [==============================] - 101s 83ms/step - loss: 8.0380 - accuracy: 0.4789
Epoch 3/3
1212/1212 [==============================] - 126s 104ms/step - loss: 8.0380 - accuracy: 0.4789


In [ ]:
#LSTM model
model = Sequential()
model.add(Embedding(input_dim=VOCAB_SIZE,
                    output_dim=128,
                    input_length=max_len))
model.add(LSTM(4, activation='tanh',return_sequences=True))
model.add(Dropout(0.15))
model.add(LSTM(40,activation='tanh',return_sequences=True))
model.add(Dropout(0.15))
model.add(LSTM(40,activation="tanh"))
model.add(Dropout(0.15))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
result = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1212/1212 [==============================] - 163s 129ms/step - loss: 0.6925 - accuracy: 0.5206 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 2/10
1212/1212 [==============================] - 155s 128ms/step - loss: 0.6924 - accuracy: 0.5207 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 3/10
1212/1212 [==============================] - 155s 128ms/step - loss: 0.6924 - accuracy: 0.5211 - val_loss: 0.6941 - val_accuracy: 0.5000
Epoch 4/10
1212/1212 [==============================] - 154s 127ms/step - loss: 0.6924 - accuracy: 0.5209 - val_loss: 0.6940 - val_accuracy: 0.5000
Epoch 5/10
1212/1212 [==============================] - 151s 125ms/step - loss: 0.6923 - accuracy: 0.5210 - val_loss: 0.6941 - val_accuracy: 0.5000
Epoch 6/10
1212/1212 [==============================] - 150s 124ms/step - loss: 0.6923 - accuracy: 0.5211 - val_loss: 0.6940 - val_accuracy: 0.5000
Epoch 7/10
1212/1212 [==============================] - 153s 126ms/step - loss: 0.6923 - accuracy: 0.5211 - val_

In [ ]:
#RNN model
model = Sequential()
model.add(Embedding(input_dim=VOCAB_SIZE,
                    output_dim=128,
                    input_length=max_len))
model.add(SimpleRNN(4, activation='tanh',return_sequences=True))
model.add(Dropout(0.15))
model.add(SimpleRNN(40,activation='tanh',return_sequences=True))
model.add(Dropout(0.15))
model.add(SimpleRNN(40,activation="tanh"))
model.add(Dropout(0.15))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
result = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))
#, input_shape=(x_train.shape[1],1)

Epoch 1/10
1212/1212 [==============================] - 98s 78ms/step - loss: 0.6608 - accuracy: 0.5810 - val_loss: 0.7604 - val_accuracy: 0.5624
Epoch 2/10
1212/1212 [==============================] - 94s 78ms/step - loss: 0.5665 - accuracy: 0.6940 - val_loss: 0.7711 - val_accuracy: 0.5851
Epoch 3/10
1212/1212 [==============================] - 95s 78ms/step - loss: 0.5428 - accuracy: 0.7145 - val_loss: 0.8067 - val_accuracy: 0.5818
Epoch 4/10
1212/1212 [==============================] - 95s 79ms/step - loss: 0.5290 - accuracy: 0.7239 - val_loss: 0.6915 - val_accuracy: 0.6000
Epoch 5/10
1212/1212 [==============================] - 94s 78ms/step - loss: 0.5168 - accuracy: 0.7314 - val_loss: 0.7702 - val_accuracy: 0.5826
Epoch 6/10
1212/1212 [==============================] - 95s 79ms/step - loss: 0.4950 - accuracy: 0.7428 - val_loss: 0.7338 - val_accuracy: 0.5937
Epoch 7/10
1212/1212 [==============================] - 94s 78ms/step - loss: 0.4789 - accuracy: 0.7569 - val_loss: 0.8367 -

In [ ]:
right = pd.DataFrame()
wrong = pd.DataFrame()
right['text'] = pd.DataFrame(data['Correct'])
wrong['text'] = pd.DataFrame(data['Incorrect'])
right['target'] = 1
wrong['target'] = 0

test_right = pd.DataFrame()
test_wrong = pd.DataFrame()
test_right['text'] = pd.DataFrame(test_data['Correct'])
test_wrong['text'] = pd.DataFrame(test_data['Incorrect'])
test_right['target'] = 1
test_wrong['target'] = 0

In [ ]:
frames = [right, wrong]
data = pd.concat(frames)
test_frames = [test_right, test_wrong]
test_data = pd.concat(test_frames)

In [ ]:
test_data

,text,target
0,In summer the people prefer to go to the beach...,1
1,It is common to see students at wherever place...,1
2,"First , I want to learn English very well , so...",1
3,I just went on an amazing trip . I took a flig...,1
4,"I like to walk in the park , in the street , n...",1
...,...,...
38291,My experience is in manager groups in a comput...,0
38292,My birthday is June 26th,0
38293,* Julie remembered that the visitors are goin...,0
38294,He was project manager for 2 years of the cust...,0


In [ ]:
final = data.dropna()
test_final = test_data.dropna()

In [ ]:
test_final

,text,target
0,In summer the people prefer to go to the beach...,1
1,It is common to see students at wherever place...,1
2,"First , I want to learn English very well , so...",1
3,I just went on an amazing trip . I took a flig...,1
4,"I like to walk in the park , in the street , n...",1
...,...,...
38291,My experience is in manager groups in a comput...,0
38292,My birthday is June 26th,0
38293,* Julie remembered that the visitors are goin...,0
38294,He was project manager for 2 years of the cust...,0


In [ ]:
#cleaning the text
stopword = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()
words = set(nltk.corpus.words.words())

def clean_text(text):
    text = text.lower()
    text = re.sub('<.*>', '', text)
    text = re.sub('\(.*\)', '', text)
    text = re.sub('\[.*\]', '', text)
    text = re.sub('{.*}', '', text)
    text = re.sub("[-+@#^/|*(){}$~`<>=_]","",text)
    text = text.replace("\\","")
    text = re.sub("\[","",text)
    text = re.sub("\]","",text)
    text = re.sub("[0-9]","",text)
    return text

In [ ]:
final['text'] = final['text'].apply(lambda x: clean_text(x))

<ipython-input-12-3a7b5cb802c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['text'] = final['text'].apply(lambda x: clean_text(x))


In [ ]:

final

,text,target
0,in my opinion the tv channels should show the ...,1
1,"there i began to study family therapy , discip...",1
2,"it seems to me , that in this song there are r...",1
3,"so , i want to say that i admire your right di...",1
4,i graduated from chung university .,1
...,...,...
153177,this is my family on the picture,0
153178,i have lunch at noon and get home p.m,0
153179,"dear mary , i listened the song that you spok...",0
153180,look at my photos of my last holidays,0


In [ ]:
tfidfvectorizer = TfidfVectorizer(max_features = 1000)
text_tfidf = tfidfvectorizer.fit_transform(final['text'])
x_train, x_test, y_train, y_test =train_test_split(text_tfidf.toarray(), final['target'], test_size = 0.3,
                                                   random_state = 42,
                                                   stratify = final['target'])


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(final['text'])
word_index = tokenizer.word_index
vocabulary = len(word_index.keys())+1

In [ ]:
#tokenization using count vectorizer
count_vectorizer = CountVectorizer()
text_countv = count_vectorizer.fit_transform(final['text'])
#text_countv.shape
#countv_result = text_countv.toarray()
#countv_result.shape

In [ ]:
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(5000, input_length = 200))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit(final['text'], final['target'], validation_data = (test_final['text'], test_final['target']),epochs = 5, )

TypeError: ignored

In [ ]:
#tokenization using tfidf vectorizer
tfidf_vectorizer = TfidfVectorizer()
text_tfidf = tfidf_vectorizer.fit_transform(final['text'])
text_tfidf.shape

(306363, 34213)

In [ ]:
#tokenization using glove
glove_dictionary = {}
with open("/content/glove.6B.300d.txt") as file:
  for each_line in file:
    words_in_line, coeff_cients = each_line.split(maxsplit=1)
    coeff_cients = np.array(coeff_cients.split(), dtype=float)
    glove_dictionary[words_in_line] = coeff_cients

embedding_matrix = np.zeros((vocabulary, 300))
for word, i in word_index.items():
  embedding_vector = glove_dictionary.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
len(glove_dictionary)

400000

In [ ]:
embedding_matrix.shape

(34577, 300)

In [ ]:
#tokenization using FastText
#FastText is an extension to Word2Vec. This breaks words into several n-grams.


In [ ]:
cv_final = final.copy(deep=True)
count_vectorizer = CountVectorizer()
for i in range(0, len(cv_final)):
  count_vectorizer.fit_transform(cv_final['text'][i])
  cv_final['text'][i] = count_vectorizer.vocabulary_